In [1]:
import random
import time
import pandas as pd
import numpy as np
from typing import Union


In [2]:
class Fibonacci:
    def recursive(self, n: int) -> int:
        if n <= 1:
            return n
        return self.recursive(n-1) + self.recursive(n-2)

    def memoizedTopDown(self, n: int, M = {}) -> int:
        if n in M:
            return M[n]
        if n <= 1:
            M[n] = n
            return n
        M[n] = self.memoizedTopDown(n-1, M) + self.memoizedTopDown(n-2, M)
        return M[n]

    def bottomUp(self, n: int) -> int:
        if n <= 1:
            return n
        M = [0] * (n+1)
        M[1] = 1
        for i in range(2, n+1):
            M[i] = M[i-1] + M[i-2]
        return M[n]

fibonacci = Fibonacci()


In [3]:
class OptimalMatrixChainMultiplication:
    def recursiveHelper(self, p: list[int], i=1, j=None, s=None) -> Union[int, list[list[int]]]:
        j = len(p) - 1 if j is None else j
        s = [[None] * len(p) for _ in range(len(p))] if s is None else s
        if i == j:
            return 0, s
        m = float('inf')
        for k in range(i, j):
            l, s = self.recursiveHelper(p, i, k, s)
            r, s = self.recursiveHelper(p, k + 1, j, s)
            q = l + r + p[i - 1] * p[k] * p[j]
            if q < m:
                m = q
                s[i][j] = k
        return m, s

    def recursive(self, p: list[int]) -> Union[int, str]:
        m, s = self.recursiveHelper(p)
        output = self.printOptimalParenthesis(s) + f" with cost {m:,}"
        return output

    def memoizedTopDownHelper(self, p: list[int], i=1, j=None, s=None, M=None) -> Union[int, list[list[int]]]:
        j = len(p) - 1 if j is None else j
        s = [[None] * len(p) for _ in range(len(p))] if s is None else s
        M = [[float('inf')] * len(p) for _ in range(len(p))] if M is None else M
        if M[i][j] != float('inf'):
            return M[i][j], s
        if i == j:
            return 0, s
        for k in range(i, j):
            l, s = self.memoizedTopDownHelper(p, i, k, s, M)
            r, s = self.memoizedTopDownHelper(p, k + 1, j, s, M)
            q = l + r + p[i - 1] * p[k] * p[j]
            if q < M[i][j]:
                M[i][j] = q
                s[i][j] = k
        return M[i][j], s

    def memoizedTopDown(self, p: list[int]) -> Union[int, str]:
        m, s = self.memoizedTopDownHelper(p)
        output = self.printOptimalParenthesis(s) + f" with cost {m:,}"
        return output

    def bottomUpHelper(self, p: list[int]) -> Union[int, list[list[int]]]:
        M = [[0 for _ in range(len(p))] for _ in range(len(p))]
        s = [[0 for _ in range(len(p))] for _ in range(len(p))]
        for l in range(2, len(p)):
            for i in range(1, len(p) - l + 1):
                j = i + l - 1
                M[i][j] = float('inf')
                for k in range(i, j):
                    q = M[i][k] + M[k + 1][j] + p[i - 1] * p[k] * p[j]
                    if q < M[i][j]:
                        M[i][j] = q
                        s[i][j] = k
        return M[1][len(p) - 1], s

    def bottomUp(self, p: list[int]) -> Union[int, str]:
        m, s = self.bottomUpHelper(p)
        output = self.printOptimalParenthesis(s) + f" with cost {m:,}"
        return output

    def printOptimalParenthesis(self, s: list[list[int]], i=1, j=None) -> str:
        j = len(s)-1 if j is None else j
        parenthisis = ""
        if i == j:
            parenthisis += f"A{i}"
        else:
            parenthisis += "("
            parenthisis += self.printOptimalParenthesis(s, i, s[i][j])
            parenthisis += self.printOptimalParenthesis(s, s[i][j] + 1, j)
            parenthisis += ")"
        return parenthisis

matrixChain = OptimalMatrixChainMultiplication()

In [4]:
algo_set = {
    'f': {
        'problem': 'Fibonacci',
        'types': {
            'r': {'label': 'Recursive', 'function': fibonacci.recursive},
            'm': {'label': 'Memoized Top Down', 'function': fibonacci.memoizedTopDown},
            'b': {'label': 'Bottom Up', 'function': fibonacci.bottomUp},
        }
    },
    'm': {
        'problem': 'Opt. Matrix Chain Multi',
        'types': {
            'r': {'label': 'Recursive', 'function': matrixChain.recursive},
            'm': {'label': 'Memoized Top Down', 'function': matrixChain.memoizedTopDown},
            'b': {'label': 'Bottom Up', 'function': matrixChain.bottomUp},
        }
    },
}

test_cases = {
    'f': [30, 35, 40, 45, 51],
    'm': [
        [random.randint(1, 100) for _ in range(11)],
        [random.randint(1, 100) for _ in range(16)],
        [random.randint(1, 100) for _ in range(18)],
        [random.randint(1, 100) for _ in range(21)],
        [random.randint(1, 100) for _ in range(23)],
    ],
}

with open('FibonacciInput.txt', 'w') as f:
    # Write each number on a new line
    for num in test_cases['f']:
        f.write(f"{num}\n")

with open('MatrixInput.txt', 'w') as f:
    # Write each matrix on a new line
    for matrix in test_cases['m']:
        f.write(f"{matrix}\n")

test_set, results = ['f', 'm'], []
print('{:>2}.  {:^23}  {:^17}  {:<15} {:>19} {:<19}'.format("No", "Problem", "Algorithm", "Input", "Elapsed time", "Result"))

for algo in test_set:
    for input in test_cases[algo]:
        for algo_type in algo_set[algo]['types'].values():
            start_time = time.time()
            output = algo_type['function'](input)
            output = f'{output:,}' if algo == 'f' else output
            duration = time.time() - start_time
            input_str = str(len(input)-1)+' matricies' if algo == 'm' else str(input)+'th fibonacci' if algo == 'f' else input
            line_new = '{:>2}.  {:^23}  {:^17}  {:<15} {:>19} {:<19}'.format(len(results)+1, algo_set[algo]['problem'], algo_type['label'], input_str, f'{duration*1000:,.3f} ms', output)
            print(line_new)
            results.append([algo_set[algo]['problem'], algo_type['label'], input_str, duration * 1000])

No.          Problem              Algorithm      Input                  Elapsed time Result             
 1.         Fibonacci             Recursive      30th fibonacci           191.066 ms 832,040            
 2.         Fibonacci         Memoized Top Down  30th fibonacci             0.012 ms 832,040            
 3.         Fibonacci             Bottom Up      30th fibonacci             0.005 ms 832,040            
 4.         Fibonacci             Recursive      35th fibonacci         2,197.958 ms 9,227,465          
 5.         Fibonacci         Memoized Top Down  35th fibonacci             0.004 ms 9,227,465          
 6.         Fibonacci             Bottom Up      35th fibonacci             0.007 ms 9,227,465          
 7.         Fibonacci             Recursive      40th fibonacci        23,515.467 ms 102,334,155        
 8.         Fibonacci         Memoized Top Down  40th fibonacci             0.005 ms 102,334,155        
 9.         Fibonacci             Bottom Up      40th f

In [5]:
# Define column names of the data frame
columns = ['Problem', 'Algorithm', 'Input', 'Runtime (ms)']

# Create a dataframe
df_original = pd.DataFrame(results, columns=columns)

df = df_original.copy()

# Format the columns to display decimal digits and add thousands separators
df['Runtime (ms)'] = df['Runtime (ms)'].apply(lambda x: f'{x:,.3f}')

# Pivot the dataframe to have 'Sort Algorithm' as columns
df = df.pivot(index=['Algorithm'], columns=['Problem', 'Input'], values=['Runtime (ms)'])

type_order = ['Recursive', 'Memoized Top Down', 'Bottom Up']
df = df.reindex(type_order, level='Type')

# Save the dataframe to a CSV and xlsx file
df.to_csv('results.csv')
df.to_excel('results.xlsx')
markdown_table = df.to_markdown(index=False)

df

Runtime (ms)                                               \
Problem                Fibonacci                                                
Input             30th fibonacci 35th fibonacci 40th fibonacci 45th fibonacci   
Algorithm                                                                       
Recursive                191.066      2,197.958     23,515.467    262,304.613   
Memoized Top Down          0.012          0.004          0.005          0.022   
Bottom Up                  0.005          0.007          0.008          0.008   

                                                                       \
Problem                          Opt. Matrix Chain Multi                
Input             51th fibonacci            10 matricies 15 matricies   
Algorithm                                                               
Recursive          4,698,324.234                   5.415    1,251.426   
Memoized Top Down          0.039                   0.144        0.408   
Bottom Up                  0.014                   0.072        0.155   

                                                            
Problem                                                     
Input             17 matricies 20 matricies   22 matricies  
Algorithm                                                   
Recursive           11,048.110  308,367.798  2,726,292.455  
Memoized Top Down        0.532        0.854          1.278  
Bottom Up                0.208        0.326          0.454

In [6]:
markdown_table = df_original.to_markdown(index=False)
print(markdown_table)

| Problem                 | Algorithm         | Input          |     Runtime (ms) |
|:------------------------|:------------------|:---------------|-----------------:|
| Fibonacci               | Recursive         | 30th fibonacci |    191.066       |
| Fibonacci               | Memoized Top Down | 30th fibonacci |      0.0119209   |
| Fibonacci               | Bottom Up         | 30th fibonacci |      0.00500679  |
| Fibonacci               | Recursive         | 35th fibonacci |   2197.96        |
| Fibonacci               | Memoized Top Down | 35th fibonacci |      0.0038147   |
| Fibonacci               | Bottom Up         | 35th fibonacci |      0.00691414  |
| Fibonacci               | Recursive         | 40th fibonacci |  23515.5         |
| Fibonacci               | Memoized Top Down | 40th fibonacci |      0.00500679  |
| Fibonacci               | Bottom Up         | 40th fibonacci |      0.00786781  |
| Fibonacci               | Recursive         | 45th fibonacci | 262305     

In [9]:
import sys

# Increase the recursion limit
sys.setrecursionlimit(10 ** 5)

with open('ExtraCredit.txt', 'w') as f:

    f.write('{:>2}.  {:^23}  {:^17}  {:<20} {:>19}\n'.format("No", "Problem", "Algorithm", "Input", "Elapsed time"))
    print('{:>2}.  {:^23}  {:^17}  {:<20} {:>19}'.format("No", "Problem", "Algorithm", "Input", "Elapsed time"))

    input = 10 ** 5
    start_time = time.time()
    fibonacci.bottomUp(input)
    duration = time.time() - start_time
    input_str = f'{input:,}'+'th fibonacci'
    line = '{:>2}.  {:^23}  {:^17}  {:<20} {:>19}\n'.format(1, "Fibonacci", "Bottom Up", input_str, f'{duration*1000:,.3f} ms')
    f.write(line)
    print(line, end='')

    input = 10 ** 4
    start_time = time.time()
    fibonacci.memoizedTopDown(input)
    duration = time.time() - start_time
    input_str = f'{input:,}'+'th fibonacci'
    line = '{:>2}.  {:^23}  {:^17}  {:<20} {:>19}\n'.format(2, "Fibonacci", "Memoized Top Down", input_str, f'{duration*1000:,.3f} ms')
    f.write(line)
    print(line, end='')

No.          Problem              Algorithm      Input                       Elapsed time
 1.         Fibonacci             Bottom Up      100,000th fibonacci           241.230 ms
 2.         Fibonacci         Memoized Top Down  10,000th fibonacci              0.006 ms
